In [ ]:
"""
6/21: To combine the edge attributes into one function

Conclusion: Not worth doing another function for it
"""

# Locally

In [ ]:
def compute_edge_attributes_locally(G,
                                          limb_obj,
                                         nodes_to_compute,
                                         edge_functions,
                                         verbose=False,
                                         directional = False,
                                         set_default_at_end = True,
                                         default_value_at_end = None,
                                         **kwargs):
    """
    Purpose: To iterate over graph edges and compute
    edge properties and store

    Pseudocode: 
    For each nodes to compute:
        get all of the edges for that node
        For each downstream partner:
            For each function:
                compute the value and store it in the edge
    Ex: 
    G = complete_graph_from_node_ids(all_branch_idx)

    nodes_to_compute = [upstream_branch]
    edge_functions = dict(sk_angle=nst.parent_child_sk_angle,
                         width_diff = nst.width_diff,
                          width_diff_percentage = nst.width_diff_percentage)

    compute_edge_attributes_between_nodes(G,
                                             nodes_to_compute,
                                             edge_functions,
                                             verbose=True,
                                             directional = False)

    """
    G = copy.deepcopy(G)
    
    if directional:
        neighbors_func = xu.downstream_nodes
    else:
        neighbors_func = xu.get_neighbors

    func_value_dict = dict()
    for n in nodes_to_compute:
        func_value_dict[n] = dict()
        down_nodes = neighbors_func(G,n)
        if verbose:
            print(f"Working on node {n}")

        for d in down_nodes:
            func_value_dict[n][d] = dict()
            if verbose:
                print(f"   Neighbor {d}")
            for func_name,func_info in edge_functions.items():
                if not callable(func_info):
                    func = func_info["function"]
                    if "arguments" in func_info.keys():
                        args = func_info["arguments"]
                    else:
                        args = dict()
                else:
                    func = func_info
                    args = dict()
                
                func_value = func(limb_obj,n,d,**args)
            
                if verbose:
                    print(f"      {func_name}: {func_value}")
                func_value_dict[n][d][func_name] = func_value
    
    xu.apply_edge_attribute_dict_to_graph(G,func_value_dict)
    
    if set_default_at_end:
        for func_name,func_info in edge_functions.items():
            d_value = default_value_at_end
            try:
                if "default_value" in func_info:
                    d_value = func_info["defualt_value"]
            except:
                pass
            xu.set_edge_attribute_defualt(G,func_name,d_value)
                
    return G

# Around Node

In [ ]:
# -------------- For node level edge attributes ------------- 3
def compute_edge_attributes_around_node(G,
                                        edge_functions,
                                        edge_functions_args = dict(),
                                        nodes_to_compute=None,
                                        verbose=False,
                                        #directional = False,
                                         set_default_at_end = True,
                                         default_value_at_end = None,
                                        **kwargs):
    """
    Purpose: To use all the edges around a node 
    to compute edge features

    """
    G = copy.deepcopy(G)
    
    if nodes_to_compute is None:
        nodes = list(G.nodes())
    
    if not nu.is_array_like(nodes_to_compute):
        nodes_to_compute = [nodes_to_compute]
        
    if verbose:
        print(f"nodes_to_compute = {nodes_to_compute}")

    for n in nodes_to_compute:
        node_edges = xu.node_to_edges(G,n)
        for func_name,func_info in edge_functions.items():
            if not callable(func_info):
                func = func_info["function"]
                if "arguments" in func_info.keys():
                    args = func_info["arguments"]
                else:
                    args = dict()
            else:
                func = func_info
                args = dict()
                
            node_edge_dict = func(G,node_edges,**args)
            
            if verbose:
                print(f"      {func_name}: {node_edge_dict}")
                
            xu.apply_edge_attribute_dict_to_graph(G,node_edge_dict,label=func_name)
            
            
    if set_default_at_end:
        for func_name,func_info in edge_functions.items():
            d_value = default_value_at_end
            try:
                if "default_value" in func_info:
                    d_value = func_info["defualt_value"]
            except:
                pass
            xu.set_edge_attribute_defualt(G,func_name,d_value)
    return G
            

# Globally

In [ ]:
def compute_edge_attributes_globally(G,
                                     edge_functions,
                                     edges_to_compute=None,
                                     verbose=False,
                                         set_default_at_end = True,
                                         default_value_at_end = None,
                                     **kwargs):
    """
    Purpose: to compute edge attributes
    that need the whole graph to be computed
    
    """
    G = copy.deepcopy(G)
    
    if edges_to_compute is None:
        edges_to_compute = xu.edges(G)
    
    other_edges_to_remove = []
    for e in edges_to_compute:
        if verbose:
            print(f"   Working on Edge {e}")
            
        for func_name,func_info in edge_functions.items():
            if not callable(func_info):
                func = func_info["function"]
                if "arguments" in func_info.keys():
                    args = func_info["arguments"]
                else:
                    args = dict()
            else:
                func = func_info
                args = dict()
                
            global_edge_dict = func(G,e[0],e[1],**args)
            
            if verbose:
                print(f"      {func_name}: {global_edge_dict}")
                
            xu.apply_edge_attribute_dict_to_graph(G,global_edge_dict,label=func_name)
            
            
    if set_default_at_end:
        for func_name,func_info in edge_functions.items():
            d_value = default_value_at_end
            try:
                if "default_value" in func_info:
                    d_value = func_info["defualt_value"]
            except:
                pass
            xu.set_edge_attribute_defualt(G,func_name,d_value)
    return G